In [21]:
import time
import numpy as np
import pandas as pd
from typing import Dict, Union
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import (
    ElementClickInterceptedException,
    NoSuchElementException,
    StaleElementReferenceException,
    TimeoutException
)
from webdriver_manager.chrome import ChromeDriverManager

# from src.common.consts import CommonConsts
# from src.modules.entities.real_estate import RealEstate
# from src.modules.repositories import RealEstateRepo



# Set up Chrome driver
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])
options.add_argument("--log-level=3")  # Suppress logs
options.add_argument("--disable-logging")  # Disable logging
options.add_argument("--silent")  # Silent mode
# options.add_argument("headless")
options.add_argument('--disable-dev-shm-usage')
options.add_argument("window-size=1920x1080")
options.add_argument("disable-gpu")
service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(options=options, service=service)
driver.set_page_load_timeout(30)
link_all = [
    "https://batdongsan.com.vn/nha-dat-ban",
    "https://batdongsan.com.vn/nha-dat-cho-thue",
]
real_estate_type = [
    "Bán",
    "Cho thuê"
]
REAL_ESTATE_TYPES = {
    "nha-dat-ban": [
        "Chung cư mini, căn hộ dịch vụ",
        "Nhà riêng",
        "Nhà biệt thự, liền kề",
        "Nhà mặt phố",
        "Shophouse, nhà phố thương mại",
        "Đất nền dự án",
        "Bán đất",
        "Condotel",
        "Kho, nhà xưởng",
        "Bất động sản khác"
    ],
    "nha-dat-cho-thue": [
        "Chung cư mini, căn hộ dịch vụ",
        "Nhà riêng",
        "Nhà biệt thự, liền kề",
        "Nhà mặt phố",
        "Nhà trọ, phòng trọ",
        "Shophouse, nhà phố thương mại",
        "Văn phòng",
        "Cửa hàng, ki ốt",
        "Kho, nhà xưởng, đất",
        "Bất động sản khác"
    ]
}

driver.get("https://batdongsan.com.vn/nha-dat-ban")

In [22]:
filter_button = driver.find_element(By.CSS_SELECTOR, "div[data-default-value='Loại nhà đất']")
filter_button.click()

all_li_elements = []

li_elements = driver.find_elements(By.CSS_SELECTOR, ".mCSB_container li[value]")
all_li_elements.extend(li_elements)

In [24]:
actions = ActionChains(driver)

li_texts = []
for li in all_li_elements:
    actions.move_to_element(li).perform()
    
    try:
        ul = li.find_element(By.CSS_SELECTOR, "ul")
        li_elems = ul.find_elements(By.CSS_SELECTOR, "li")
        for li_elem in li_elems:
            print(li_elem.text)
            li_texts.append(li_elem.text)
    except NoSuchElementException:
        li_texts.append(li.text)
        print(li.text)
    print("-" * 50)
    
    # li_texts.append(li.text)


print(li_texts)

Chung cư mini, căn hộ dịch vụ
--------------------------------------------------
Chung cư mini, căn hộ dịch vụ
--------------------------------------------------
Nhà riêng
Nhà biệt thự, liền kề
Nhà mặt phố
Shophouse, nhà phố thương mại
--------------------------------------------------
Nhà riêng
--------------------------------------------------
Nhà biệt thự, liền kề
--------------------------------------------------
Nhà mặt phố
--------------------------------------------------
Shophouse, nhà phố thương mại
--------------------------------------------------
Đất nền dự án
Bán đất
--------------------------------------------------
Đất nền dự án
--------------------------------------------------
Bán đất
--------------------------------------------------
Condotel
--------------------------------------------------
Condotel
--------------------------------------------------
Kho, nhà xưởng
--------------------------------------------------
Bất động sản khác
----------------------------------

In [25]:
li_texts = list(set(li_texts))
li_texts

['Chung cư mini, căn hộ dịch vụ',
 'Shophouse, nhà phố thương mại',
 'Đất nền dự án',
 'Nhà biệt thự, liền kề',
 'Bất động sản khác',
 'Nhà mặt phố',
 'Condotel',
 'Kho, nhà xưởng',
 'Bán đất',
 'Nhà riêng']

In [15]:
actions = ActionChains(driver)

last_height = len(li_elements)
while True:
    # Scroll to the last element in the current list
    actions.move_to_element(li_elements[-1]).perform()
    
    # Wait for new elements to load
    time.sleep(1)
    
    # Get the updated list of li elements
    li_elements = driver.find_elements(By.CSS_SELECTOR, ".mCSB_container li[value]")
    
    # If the number of elements doesn't change, we have reached the end
    if len(li_elements) == last_height:
        break
    
    # Update the last_height and extend the all_li_elements list
    last_height = len(li_elements)
    all_li_elements.extend(li_elements)

# Remove duplicates from all_li_elements (since they might have been added multiple times)
unique_li_elements = list(set(all_li_elements))

# Print the value and text of each unique li element
for elem in unique_li_elements:
    print(elem.get_attribute("value"))
    print(elem.text)

575
Shophouse, nhà phố thương mại
48
Bất động sản khác
324

361
Các loại đất bán
Đất nền dự án
Bán đất
325

44
Trang trại, khu nghỉ dưỡng
Condotel
650

163

41

562
Condotel
45
Kho, nhà xưởng
40
Đất nền dự án
362
Shophouse, nhà phố thương mại
283
Bán đất


In [6]:
# find .mCSB_container li with innerText = "Kho, nhà xưởng"
filter_dropdown = driver.find_element(By.CSS_SELECTOR, ".mCSB_container")
filter_dropdown.text

'Shophouse, nhà phố thương mại\nCác loại đất bán\nĐất nền dự án\nBán đất\nTrang trại, khu nghỉ dưỡng\nCondotel\nKho, nhà xưởng\nBất động sản khác'

In [8]:
# Get all elem that are li tag
elems = driver.find_elements(By.CSS_SELECTOR, ".mCSB_container li[value]")
re_types = []
for elem in elems:
        print(elem.get_attribute("value"))
        print(elem.text)
        # re_types.append(elem.text)
        # try:
        #     ul = elem.find_element(By.CSS_SELECTOR, "ul")
        #     li_elems = ul.find_elements(By.CSS_SELECTOR, "li")
        #     for li_elem in li_elems:
        #         print(f"--li: {li_elem.text}")
        #     print(f"--ul ")
        # except NoSuchElementException:
        #     ul = None
        #     print(f"--no ul")
        # print("-" * 50)
re_types


324
Căn hộ chung cư
Chung cư mini, căn hộ dịch vụ
650
Chung cư mini, căn hộ dịch vụ
362
Các loại nhà bán
Nhà riêng
Nhà biệt thự, liền kề
Nhà mặt phố
Shophouse, nhà phố thương mại
41
Nhà riêng
325
Nhà biệt thự, liền kề
163
Nhà mặt phố
575
Shophouse, nhà phố thương mại
361

40

283

44

562

45

48



[]